# TEXTBLOB DEMO

In [0]:
import textblob

In [0]:
from textblob import TextBlob

In [0]:
p="It's raining very heavily in Mumbai.Rainfall in India is at a high"

In [0]:
blob=TextBlob(p)

In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
blob.words

WordList(['It', "'s", 'raining', 'very', 'heavily', 'in', 'Mumbai.Rainfall', 'in', 'India', 'is', 'at', 'a', 'high'])

In [0]:
blob.sentences[0]

Sentence("It's raining very heavily in Mumbai.Rainfall in India is at a high")

In [0]:
blob.sentiment

Sentiment(polarity=-0.05, subjectivity=0.595)

In [0]:
p2="Terrorism is ravaging the nation"
blob=TextBlob(p2)
blob.sentiment

Sentiment(polarity=0.0, subjectivity=0.0)

In [0]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
blob.tags

[('Terrorism', 'NN'),
 ('is', 'VBZ'),
 ('ravaging', 'VBG'),
 ('the', 'DT'),
 ('nation', 'NN')]

In [0]:
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [0]:
for x in blob.noun_phrases:
  print(x)

terrorism


# SPAM-HAM

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("spam.csv",encoding="ISO-8859-1")

In [5]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [6]:
remove=pd.read_csv("stopwords.txt",header=None)

In [7]:
remove.head()

,0
0,a
1,about
2,above
3,across
4,after


In [8]:
remove.shape

(426, 1)

In [9]:
from string import punctuation
from nltk.corpus import stopwords

In [10]:
stopword=remove[0].tolist()+list(punctuation)+list(stopwords.words("english"))

In [11]:
from nltk.tokenize import word_tokenize
corpus=df["v2"].str.lower().tolist()

In [12]:
from nltk.stem import LancasterStemmer
lan=LancasterStemmer()

In [13]:
final_corpus=[]
for i in range(len(corpus)):
    text=word_tokenize(corpus[i])
    text=[lan.stem(t) for t in text if t not in stopword]
    text=" ".join(text)
    final_corpus.append(text)

In [15]:
new_df=pd.DataFrame(final_corpus)

In [16]:
from sklearn.preprocessing import LabelEncoder

In [17]:
label=LabelEncoder()

In [18]:
y=label.fit_transform(df["v1"])

In [19]:
y=pd.DataFrame(y)

In [20]:
new_df["y"]=y

In [22]:
new_df.columns=["mail","label"]

In [23]:
new_df.head()

,mail,label
0,jurong crazy.. avail bug world la buffet ... c...,0
1,ok lar ... jok wif on ...,0
2,fre entry 2 wkly comp win fa cup fin tkts 21st...,1
3,dun hor ... ...,0
4,nah n't goe usf liv,0


In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
vec=TfidfVectorizer()
vector=vec.fit_transform(new_df["mail"])

In [26]:
new_df.shape

(5572, 2)

In [27]:
x_train = vector[:4572].toarray() 
y_train = new_df.iloc[:4572,1]
x_test = vector[4572:].toarray() 
y_test = new_df.iloc[4572:,1]

In [29]:
from sklearn.linear_model import LogisticRegression

In [30]:
logreg=LogisticRegression()
model=logreg.fit(x_train,y_train)
y_pred = model.predict(x_test)

C:\Users\Ruhi\Anaconda3_new\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [31]:
from sklearn.metrics import confusion_matrix

In [33]:
print(confusion_matrix(y_test,y_pred))

[[866   1]
 [ 47  86]]


In [34]:
from sklearn.metrics import classification_report

In [35]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97       867
           1       0.99      0.65      0.78       133

   micro avg       0.95      0.95      0.95      1000
   macro avg       0.97      0.82      0.88      1000
weighted avg       0.95      0.95      0.95      1000

